In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')



import random
import math
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(config.seed)

In [ ]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [ ]:
transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

transform_validation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

transform  = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])


In [ ]:
trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=True,
                                        transform=transform_train)

vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=True,
                                        transform=transform_validation)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/images/Statistical_Deep_Image",
                                           transform = transform)

In [ ]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 64,
                          shuffle=True,
                          num_workers=2)


test_loader = DataLoader(testset,
                          batch_size=32,
                          shuffle=False,
                          num_workers=2)

모델 넣기 

In [ ]:
!pip install timm
import timm


timm.list_models()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 548 kB 14.8 MB/s 
     |████████████████████████████████| 163 kB 70.0 MB/s 


['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw

In [ ]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return probs

In [ ]:
net = LeNet5(n_classes = 10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

total=0 
correct = 0

In [ ]:
for epoch in range(50):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0

  with torch.no_grad():
    val_loss = 0.0
    for k, data1 in enumerate(vaild_loader, 0):
      val_inputs, val_label = data1[0].to(device), data1[1].to(device)
      val_output = net(val_inputs)
      v_loss = criterion(val_output, val_label)
      val_loss += v_loss
  print("validation loss {}".format(val_loss))

  with torch.no_grad():
    for data in test_loader:
      images, labels = data[0].to(device), data[1].to(device)
      outputs= net(images)
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print("testset Accuracy  : {}".format(100* correct/total))
  total=0
  correct=0

Epoch: 1,Batch : 750, Loss:0.8632899314165116
Epoch: 1,Batch : 1500, Loss:0.8623618266582489
validation loss 360.4013366699219
10.65
Epoch: 2,Batch : 750, Loss:0.8578474785089493
Epoch: 2,Batch : 1500, Loss:0.8472509235143661
validation loss 352.2631530761719
8.1
Epoch: 3,Batch : 750, Loss:0.8407577077150344
Epoch: 3,Batch : 1500, Loss:0.8377511608600616
validation loss 349.51251220703125
7.45
Epoch: 4,Batch : 750, Loss:0.8345701067447663
Epoch: 4,Batch : 1500, Loss:0.8298817946910858
validation loss 346.21734619140625
6.9
Epoch: 5,Batch : 750, Loss:0.8265466728210449
Epoch: 5,Batch : 1500, Loss:0.8238540161848068
validation loss 343.687744140625
7.6
Epoch: 6,Batch : 750, Loss:0.8210046716928482
Epoch: 6,Batch : 1500, Loss:0.817640862941742
validation loss 340.8033447265625
7.6
Epoch: 7,Batch : 750, Loss:0.8141855626702309
Epoch: 7,Batch : 1500, Loss:0.8110812964439392
validation loss 338.1292724609375
8.4
Epoch: 8,Batch : 750, Loss:0.8078433132171631
Epoch: 8,Batch : 1500, Loss:0.8064

In [ ]:
for epoch in range(50,150):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0

  with torch.no_grad():
    val_loss = 0.0
    for k, data1 in enumerate(vaild_loader, 0):
      val_inputs, val_label = data1[0].to(device), data1[1].to(device)
      val_output = net(val_inputs)
      v_loss = criterion(val_output, val_label)
      val_loss += v_loss
  print("validation loss {}".format(val_loss))

  with torch.no_grad():
    for data in test_loader:
      images, labels = data[0].to(device), data[1].to(device)
      outputs= net(images)
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print("testset Accuracy  : {}".format(100* correct/total))
  total=0
  correct=0

Epoch: 51,Batch : 750, Loss:0.7259917783141137
Epoch: 51,Batch : 1500, Loss:0.7258416210412979
validation loss 307.23040771484375
testset Accuracy  : 16.65
Epoch: 52,Batch : 750, Loss:0.7250983639359474
Epoch: 52,Batch : 1500, Loss:0.7256888750791549
validation loss 305.6312255859375
testset Accuracy  : 15.2
Epoch: 53,Batch : 750, Loss:0.7240996459722518
Epoch: 53,Batch : 1500, Loss:0.7243660908341408
validation loss 305.6039123535156
testset Accuracy  : 15.35
Epoch: 54,Batch : 750, Loss:0.7251142844557762
Epoch: 54,Batch : 1500, Loss:0.7225515919923783
validation loss 305.4737548828125
testset Accuracy  : 15.75
Epoch: 55,Batch : 750, Loss:0.7225439749956131
Epoch: 55,Batch : 1500, Loss:0.723250226020813
validation loss 304.8063659667969
testset Accuracy  : 16.1
Epoch: 56,Batch : 750, Loss:0.7223725637793541
Epoch: 56,Batch : 1500, Loss:0.7225008317232132
validation loss 305.226318359375
testset Accuracy  : 15.45
Epoch: 57,Batch : 750, Loss:0.7209217605590821
Epoch: 57,Batch : 1500, Lo

In [ ]:
for epoch in range(150,1000):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 1500 ==1499:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0
  if epoch % 10 == 0:
    with torch.no_grad():
      for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs= net(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print("testset Accuracy  : {}".format(100* correct/total))
    total=0
    correct=0

Epoch: 151,Batch : 1500, Loss:1.356152800142765
testset Accuracy  : 24.55
Epoch: 152,Batch : 1500, Loss:1.354506415605545
Epoch: 153,Batch : 1500, Loss:1.353373305439949
Epoch: 154,Batch : 1500, Loss:1.3540266038179398
Epoch: 155,Batch : 1500, Loss:1.3525650131106377
Epoch: 156,Batch : 1500, Loss:1.3522066872119904
Epoch: 157,Batch : 1500, Loss:1.3524565351605415
Epoch: 158,Batch : 1500, Loss:1.3511219604611397
Epoch: 159,Batch : 1500, Loss:1.351419348180294
Epoch: 160,Batch : 1500, Loss:1.3506899450421332
Epoch: 161,Batch : 1500, Loss:1.3497425371408462
testset Accuracy  : 22.9
Epoch: 162,Batch : 1500, Loss:1.3496876139640808
Epoch: 163,Batch : 1500, Loss:1.3486166974902154
Epoch: 164,Batch : 1500, Loss:1.348358838737011
Epoch: 165,Batch : 1500, Loss:1.3475145441293717
Epoch: 166,Batch : 1500, Loss:1.3477143142819406
Epoch: 167,Batch : 1500, Loss:1.3456502810716628
Epoch: 168,Batch : 1500, Loss:1.3450240722298623
Epoch: 169,Batch : 1500, Loss:1.344456161737442
Epoch: 170,Batch : 1500,

pretrained 넣기


In [ ]:
PATH = '/content/drive/MyDrive/share/trained_model/lenet5.pth'



In [ ]:
torch.save(net.state_dict(), PATH)

In [ ]:
classes = ('plane','automobile','bird','cat','deer',
           'dog','frog','horse','ship','truck')

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    c= (predicted == labels).squeeze()
    for i in range(4):
      label= labels[i]
      class_correct[label]+= c[i].item()
      class_total[label]+= 1

for i in range(10):
  print("Accuracy of {} : {} %".format(classes[i],100* class_correct[i]/class_total[i]))

Accuracy of plane : 14.285714285714286 %
Accuracy of automobile : 16.666666666666668 %
Accuracy of bird : 12.5 %
Accuracy of cat : 4.166666666666667 %
Accuracy of deer : 21.428571428571427 %
Accuracy of dog : 8.333333333333334 %
Accuracy of frog : 12.5 %
Accuracy of horse : 4.166666666666667 %
Accuracy of ship : 21.428571428571427 %
Accuracy of truck : 20.833333333333332 %


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in train_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("trainset Accuracy  : {}".format(100* correct/total))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("vaildset Accuracy  : {}".format(100* correct/total))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("testset Accuracy  : {}".format(100* correct/total))